In [3]:
import os

def get_filenames(folder_path):
    filenames = []
    for file in os.listdir(folder_path):
        if file.endswith(".jpg") or file.endswith(".tiff"):
            filenames.append(os.path.join(folder_path, file))
    return filenames

folder_path = "./DANE/przykladowe_zdj/przykladowe_zdj/"
jpg_filenames = get_filenames(folder_path)
print(jpg_filenames)

['./DANE/przykladowe_zdj/przykladowe_zdj/0a16d521-1f33-45be-bde8-376c9026fd96.tiff', './DANE/przykladowe_zdj/przykladowe_zdj/0a1d5be5-8487-4733-babb-c1c71cee876b.tiff', './DANE/przykladowe_zdj/przykladowe_zdj/0a2d8a84-07ee-4d76-a246-8625d1c2cc65.jpg', './DANE/przykladowe_zdj/przykladowe_zdj/0a3e5c11-9f35-4398-9598-da5177296461.tiff', './DANE/przykladowe_zdj/przykladowe_zdj/0a6bfb5d-a4d7-4e4c-a1ba-22f259078b3d.tiff', './DANE/przykladowe_zdj/przykladowe_zdj/0a7cd32d-6127-433c-b122-a6eb30a53f9e.tiff', './DANE/przykladowe_zdj/przykladowe_zdj/0ad15aa1-78bf-4e4f-86c4-f0c395785e38.tiff', './DANE/przykladowe_zdj/przykladowe_zdj/0b645a67-6f38-4da2-95e2-7460f800c275.tiff', './DANE/przykladowe_zdj/przykladowe_zdj/0b7397d3-6cbf-4616-b5eb-44022be04a63.tiff']


In [12]:
import cv2
import pytesseract
from pytesseract import Output
import pandas as pd

def detect_image_table(filepaths):
    path_to_tesseract = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    pytesseract.pytesseract.tesseract_cmd = path_to_tesseract  # Change this path to your Tesseract executable

    def is_image_or_table(image):
        d = pytesseract.image_to_data(image, output_type=Output.DICT)
        has_image, has_table = False, False
        for i in range(len(d['level'])):
            if d['level'][i] == 2:  # Level 2 corresponds to block-level elements
                print(d['block_num'][i])
                if d['block_num'][i] != 0:
                    if d['block_num'][i] != prev_block_num:

                        if d['text'][i] == "":
                            has_image = True
                        elif d['conf'][i] < 100:
                            has_table = True
            prev_block_num = d['block_num'][i]
        return has_image, has_table

    data = []
    for filepath in filepaths:
        img = cv2.imread(filepath)
        has_image, has_table = is_image_or_table(img)
        data.append([filepath, has_image, has_table])

    df = pd.DataFrame(data, columns=['filepath', 'hasImage', 'hasTable'])
    return df

filepaths = jpg_filenames
result_df = detect_image_table(filepaths)
print(result_df)

1
2
3
4
1
2
3
4
5
6
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
12
13
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
1
2
3
4
5
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
                                            filepath  hasImage  hasTable
0  ./DANE/przykladowe_zdj/przykladowe_zdj/0a16d52...      True     False
1  ./DANE/przykladowe_zdj/przykladowe_zdj/0a1d5be...      True     False
2  ./DANE/przykladowe_zdj/przykladowe_zdj/0a2d8a8...      True     False
3  ./DANE/przykladowe_zdj/przykladowe_zdj/0a3e5c1...      True     False
4  ./DANE/przykladowe_zdj/przykladowe_zdj/0a6bfb5...      True     False
5  ./DANE/przykladowe_zdj/przykladowe_zdj/0a7cd32...      True     False
6  ./DANE/przykladowe_zdj/przykladowe_zdj/0ad15aa...      True     False
7  ./DANE/przykladowe_zdj/przykladowe_zdj/0b645a6...      True     False
8  ./DANE/przykladowe_zdj/pr